In [1]:
!pip install transformers
!pip install nltk
import nltk
nltk.download("punkt")
nltk.download("stopwords")

     |████████████████████████████████| 2.6 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 60.1 MB/s 
     |████████████████████████████████| 636 kB 47.9 MB/s 
     |████████████████████████████████| 895 kB 60.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import transformers
import shutil
from torch.utils.data import DataLoader
from nltk.corpus import stopwords
import os

In [3]:
!unzip "/content/drive/MyDrive/Colab Notebooks/EHCR Dataset.zip"

Streaming output truncated to the last 5000 lines.
  inflating: test_violations/Article6/001-57806.txt  
  inflating: test_violations/Article6/001-57807.txt  
  inflating: test_violations/Article6/001-57809.txt  
  inflating: test_violations/Article6/001-57810.txt  
  inflating: test_violations/Article6/001-57813.txt  
  inflating: test_violations/Article6/001-57814.txt  
  inflating: test_violations/Article6/001-57815.txt  
  inflating: test_violations/Article6/001-57820.txt  
  inflating: test_violations/Article6/001-57832.txt  
  inflating: test_violations/Article6/001-57838.txt  
  inflating: test_violations/Article6/001-57841.txt  
  inflating: test_violations/Article6/001-57842.txt  
  inflating: test_violations/Article6/001-57849.txt  
  inflating: test_violations/Article6/001-57850.txt  
  inflating: test_violations/Article6/001-57851.txt  
  inflating: test_violations/Article6/001-57858.txt  
  inflating: test_violations/Article6/001-57859.txt  
  inflating: test_violations/Ar

In [5]:
# Defining a function to pre-process a given case report
def preprocess(text):
  if text.find("AS TO THE LAW") != -1:
    ind = text.find("AS TO THE LAW")
  else:
    ind = text.find("THE LAW")
  text = text[0:ind]
  text = text.lower().split()
  stop_words = stopwords.words("english")
  pre_processed = []
  for word in text:
    if word not in stop_words and word.isalnum():
      pre_processed.append(word)
  return pre_processed

# Readinig in the training data
training_dir = "/content/train"
# training_dir = "./Legal Data/train"
df = pd.DataFrame(columns = ["case_report", "article_violated"])
for dir in os.listdir(training_dir):
  for subdir in ["violation", "non-violation"]:
    base_path = training_dir + "/" + dir + "/" + subdir
    for filename in os.listdir(base_path):
      with open(base_path + "/" + filename, 'r', encoding="utf-8") as file:
        case_text = file.read().replace('\n', '')
      case_text = preprocess(case_text)
      if subdir == "violation":
        violated = int(dir[len("Article"):])
      else:
        violated = 0
      case_data = {
          "case_report": case_text,
          "article_violated": violated,
      }
      df = df.append(case_data, ignore_index = True)

In [ ]:
# Exporting preprocessed training in CSV format for later use
# df.to_csv("/content/drive/MyDrive/Colab Notebooks/EHCR_training.csv")

In [6]:
# Making a train-test split, training set defined as 90% of overall training data 
# Validation set defined as 10%
# Test set defined as 20% of overall data
training_set = df.sample(frac = 0.9, random_state = 42)
validation_set = df.drop(training_set.index)
training_cases = training_set["case_report"].to_list()
training_labels = training_set["article_violated"].to_list()
validation_cases = validation_set["case_report"].to_list()
validation_labels = validation_set["article_violated"].to_list()

In [7]:
model_name = "distilbert-base-uncased"

from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from torch.utils.data import Dataset

# Initializing a DistilBert Model and Tokenizer
Tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
Model = DistilBertForSequenceClassification.from_pretrained(model_name)

class EHCR_Dataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

# Converting training and validation corpuses into their respective encodings
training_encodings = Tokenizer(training_cases, padding = True, truncation = True, is_split_into_words=True)
validation_encodings = Tokenizer(validation_cases, padding = True, truncation = True, is_split_into_words=True)
training_dataset = EHCR_Dataset(training_encodings, training_labels)
validation_dataset = EHCR_Dataset(validation_encodings, validation_labels)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [1]:
from transformers import Trainer, TrainingArguments

# Defining a set of training arguments
training_args = TrainingArguments(
    output_dir = "./results",
    learning_rate = 0.002,
    weight_decay = 0.01,
    num_train_epochs = 5,
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 16
)

# Defining a trainer function
trainer = Trainer(
    model = Model,
    args = training_args,
    train_dataset = training_dataset,
    eval_dataset = validation_dataset
)

trainer.train()

NameError: ignored

In [ ]:
from transformers import AdamW
from torch.utils.data import DataLoader

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
Model.to(device)
Model.train()
loader = DataLoader(training_dataset, batch_size = 5, shuffle = True)
optim = AdamW(Model.parameters(), lr = 0.002)
num_epochs = 5
for i in range(num_epochs):
  for batch in loader:
    optim.zero_grad()
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    outputs = Model(input_ids, attention_mask = attention_mask, labels = labels)
    loss = outputs[0]
    loss.backward()
    optim.step()
  print(loss)

Model.eval()